## 关于这个笔记本

该笔记本由 B220400 班某咸鱼创建，用于自动化处理“原子光谱的定性研究”（实际上是 P84 2.10.4 用分光计研究光栅光谱）实验的数据。

数据请填入 `he.csv` 和 `hg.csv`。

注：该表格将一个角度数值分为两列以方便填入。例如，对于 $250\degree30'$ 这样一个角度数据，第一列填入 `250`，第二列填入 `30`，笔记本会自动生成最终的角度数据 `250.5`。

In [ ]:
HE_DATA_FILE = 'he.csv'
HG_DATA_FILE = 'hg.csv'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## He

In [ ]:
he_useful_features = ['Color', '$\\lambda$', '$\\theta_l$', '$\\theta_r$', '$\\theta\'_l$', '$\\theta\'_r$', '$\\theta$', '$\\sin{\\theta}$']

he_data = pd.read_csv(HE_DATA_FILE)

he_data['$\\theta_l$'] = he_data['$\\theta_l(\degree)$'] + he_data['$\\theta_l(\')$'] / 60
he_data['$\\theta_r$'] = he_data['$\\theta_r(\degree)$'] + he_data['$\\theta_r(\')$'] / 60
he_data['$\\theta\'_l$'] = he_data['$\\theta\'_l(\degree)$'] + he_data['$\\theta\'_l(\')$'] / 60
he_data['$\\theta\'_r$'] = he_data['$\\theta\'_r(\degree)$'] + he_data['$\\theta\'_r(\')$'] / 60

he_data['$\\theta$'] = (np.abs(he_data['$\\theta\'_l$']-he_data['$\\theta_l$']) + np.abs(he_data['$\\theta\'_r$']-he_data['$\\theta_r$'])) / 4
he_data['$\\sin{\\theta}$'] = np.sin(he_data['$\\theta$']*np.pi/180)

he_data[he_useful_features]

In [ ]:
X = np.array(he_data['$\\sin{\\theta}$'])[:, np.newaxis]
y = np.array(he_data['$\\lambda$'])

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)

prediction_line_label = f'$\\lambda={float(model.coef_)}' + '\\sin{\\theta}$' # + f'+{float(model.intercept_)}'
lambda_sin_theta_plot = sns.regplot(x = X, y=y, ci=None, line_kws={'label':prediction_line_label})
lambda_sin_theta_plot.set_title('Figure  $\\lambda-\\sin{\\theta}$')
lambda_sin_theta_plot.set_xlabel('$\\sin{\\theta}$')
lambda_sin_theta_plot.set_ylabel('$\\lambda/(nm)$')
lambda_sin_theta_plot.legend()

In [ ]:
grating_constant_d = float(model.coef_)
print(f'd = {grating_constant_d}')

## Hg

In [ ]:
hg_useful_features = ['Color', '$\\lambda$', '$\\theta_l$', '$\\theta_r$', '$\\theta\'_l$', '$\\theta\'_r$', '$\\theta$', '$\\sin{\\theta}$', '$\\lambda_{o}$', '$\\eta$']

hg_data = pd.read_csv(HG_DATA_FILE)

hg_data['$\\theta_l$'] = hg_data['$\\theta_l(\degree)$'] + hg_data['$\\theta_l(\')$'] / 60
hg_data['$\\theta_r$'] = hg_data['$\\theta_r(\degree)$'] + hg_data['$\\theta_r(\')$'] / 60
hg_data['$\\theta\'_l$'] = hg_data['$\\theta\'_l(\degree)$'] + hg_data['$\\theta\'_l(\')$'] / 60
hg_data['$\\theta\'_r$'] = hg_data['$\\theta\'_r(\degree)$'] + hg_data['$\\theta\'_r(\')$'] / 60

hg_data['$\\theta$'] = (np.abs(hg_data['$\\theta\'_l$']-hg_data['$\\theta_l$']) + np.abs(hg_data['$\\theta\'_r$']-hg_data['$\\theta_r$'])) / 4
hg_data['$\\sin{\\theta}$'] = np.sin(hg_data['$\\theta$']*np.pi/180)
hg_data['$\\lambda_{o}$'] = hg_data['$\\sin{\\theta}$'] * grating_constant_d
hg_data['$\\eta$'] = np.abs(hg_data['$\\lambda_{o}$'] - hg_data['$\\lambda$']) / hg_data['$\\lambda$'] * 100

hg_data[hg_useful_features]